# Présentation de la boîte à outil Julia pour l'algèbre Max-Plus

L'algebre Max-Plus (max+) redéfinit les opérateurs addition et multiplication de l'algèbre classique par respectivement les opérateurs maximum et addition :

$$a \oplus b \triangleq \max(a,b)$$
$$a \otimes b \triangleq a + b$$

L'algebre Max-Plus permet de modéliser et d'évaluer les performances de systèmes à évènements discrets (réseaux de transport ou de télécom, systèmes de production), mais également dans des domaines tels que la théorie de la décision, recherche opérationnelle ...

Une boîte à outil Max-Plus pour le langage Julia version >= 1.0.3 peut être téléchargée depuis ce repo GitHub https://github.com/Lecrapouille/MaxPlus.jl ou bien depuis le système de paquets de Julia via la commande `] add MaxPlus`. Ce projet est un portage de la boîte à outil qui état intégrée dans le logiciel Scilab, puis par la suite, dans le logiciel [ScicosLab](http://www.scicoslab.org), un fork maintenu à l'époque par les anciens chercheurs de Scilab mais qui est désormais remplacé par le second fork [NSP](https://cermics.enpc.fr/~jpc/nsp-tiddly/mine.html).

Ce document permet à la fois d'introduire l'algèbre Max-Plus tout en présentant les fonctions de base de cette boîte à outils Julia. Pour ceux qui maîtrisent déjà cette algèbre, peuvent aller directement consulter cette [bibliographie](../docs/src/bibliography.md). Pour ceux qui désirent comparer les résultats obtenus avec ceux fournis par Sicoslab on rappellera qu'un nombre Max-Plus Sicoslab est créé avec la notation suivante `#(42)`, que les éléments neutres et absorbants obtenus par `%0` et `%1` et qu'une démonstration interactive de la boîte à outil peut être lancée depuis leur menu.

## Installation de la boîte à outil Julia

Tout d'abord vérifions la version de votre Julia. Cette boîte à outils Max-Plus fonctionne doit fonctionner avec toutes les versions de Julia même celles obsolètes (v0.4, v0.7). Certaines versions de Julia apportent des correctifs (comme sur les matrices creuses: v1.3), d'autres ajoutent des régressions (matrice identité: v1.0; produit matriciel matrice creuse avec matrice pleine: v1.4). Certains bugs sont vieux et ne sont pas corrigés (correctifs non pris en compte) mais ils seront appliqués avec ce paquet Max-Plus. Pour plus d'information, confère le fichier `fallbacks.jl`.

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: AMD Ryzen 7 1800X Eight-Core Processor
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, znver1)


Ensuite, nous allons installer la boîte à outils Max-Plus Julia. Vous avez plusieurs options pour le faire. Les codes suivants ne fonctionnent pas directement depuis ce document Jupyter, pensez donc à les décommenter et de les exécuter depuis le mode interactif Julia (REPL) :
- vous pouvez télécharger la version du code source depuis GitHub :

In [ ]:
# using Pkg; Pkg.add(PackageSpec(url="https://github.com/Lecrapouille/MaxPlus.jl"));
# using MaxPlus;

- Soit depuis le gestionnaire de paquet Julia (version stable):

In [ ]:
# using Pkg; Pkg.add("MaxPlus")

## Utiliser Max-Plus depuis un document Jupyter

Une fois le paquet Julia Max-Plus installé depuis le REPL Julia, lancez un notebook Jupyter et ouvrez cette page ci via les commandes REPL suivantes :

In [ ]:
# using IJulia
# notebook()

Depuis ce document Jupyter (celui que vous lisez), chargez la boîte à outil Max-Plus depuis le dossier MaxPlus.jl :

In [2]:
push!(LOAD_PATH, pwd())
using MaxPlus

┌ Info: Precompiling MaxPlus [41177cfe-c387-11e9-2806-edd030e4594e]
└ @ Base loading.jl:1317


Pour le moment et dans un soucis pédagogique on active un mode d'affichage particulier des nombres Max-Plus. L'explication sera donnée plus tard.

In [3]:
# Force l'affichage des nombres Max-Plus pour afficher les -infini
mp_change_display(0);

Sur Jupyter le mode $\LaTeX$ semble être forcé, hors cette boîte à outils permet d'afficher les matrices Max-Plus en code $\LaTeX$ ce qu'on ne veut pas ! On force donc l'affichage en texte plein.

In [4]:
Base.show(io::IO, ::MIME"text/latex", x::MP{T}) where T = show(io, MIME"text/plain", x)
Base.show(io::IO, ::MIME"text/latex", A::ArrMP{T}) where T = show(io, MIME"text/plain", A)
Base.show(io::IO, ::MIME"text/latex", S::SpaMP{T}) where T = show(io, MIME"text/plain", S)

## Les types Max-Plus pour Julia

Avant de présenter l'algèbre Max-Plus, créons quelques nombres scalaires Max-Plus sous Julia. Par exemple, on écrira :

In [5]:
a = MP(1.0)
b = MP(3.0)
c = MP(5)
typeof(a), typeof(b), typeof(c)

(MP{Float64}, MP{Float64}, MP{Int64})

Les nombres max+ sont en fait templatés (Float ou Int) mais Julia déduit directement leur type. On peut éventuellement forcer leur type : 

In [6]:
a = MP{Float64}(42)
a, typeof(a)

(42.0, MP{Float64})

Même idée pour les entiers (bien que les entiers soient moins recommandés) :

In [7]:
b = MP{Int64}(3)
b, typeof(b)

(3, MP{Int64})

Les nombres max+ contaminent sur les autres nombres (entiers, réels) : ils convertissent un nombre non max+ en nombre max+ via les opérateurs arithmétiques où opérateurs de promotion implicites :

In [8]:
d = 1.0
typeof(c), typeof(d), typeof(c + d)

(MP{Int64}, Float64, MP{Float64})

Nous voyons que l'addition Max-Plus à transformé la variable `c` de type MP{Int64} en type MP{Float64}. Même idée pour les nombres entiers :

In [9]:
f = 1
typeof(c), typeof(f), typeof(c + f)

(MP{Int64}, Int64, MP{Int64})

La contamination fonctionne également sur les matrices denses et creuses :

In [10]:
[MP(1.0) 2; 3 4]

2×2 Matrix{MP{Float64}}:
  1.0   2.0
  3.0   4.0


MP(1.0) de type MP{Float64} a contaminé les nombres entiers *classiques* 2 3 et 4 en MP{Float64}.

Voici une autre façon de le faire :

In [11]:
MP([1.0 2; 3 4])

2×2 Matrix{MP{Float64}}:
  1.0   2.0
  3.0   4.0


In [17]:
mpsparse([1.0 2; 3 4])

2×2 SparseMatrixCSC{MP{Float64}, Int64} with 4 stored entries:
  1   2
  3   4


Voici un autre exemple plus complexe de matrice contenant des nombres infinis $-\infty$ (qui sont du type MP{Float}):

In [12]:
I = MP([-Inf 0; 0 -Inf])

2×2 Matrix{MP{Float64}}:
  -Inf    0.0
   0.0   -Inf


In [18]:
# FIXME
J = mpsparse([-Inf 0; 0 -Inf])
J.nzval

2×2 SparseMatrixCSC{MP{Float64}, Int64} with 2 stored entries:
  .   .
  .   .


Si l'on désire convertir un nombre Max-Plus en un nombre ordinaire (au sens algèbre classique, celle que l'on utilise tous les jours) :

In [13]:
b = plustimes(a)
typeof(a), typeof(f), typeof(MP(b)), typeof(plustimes(MP(b)))

(MP{Float64}, Int64, MP{Float64}, Float64)

Nous voyons que la variable `a` étant du type MP{Float64}, `b` est devenu de type Float64. Elle se comportera désormais comme un nombre de l'algèbre classique.

In [14]:
[f       a
 f + f   a + a]

2×2 Matrix{MP{Float64}}:
  1.0   42.0
  2.0   42.0


Vu que `a` est de MP{Float64} il contamine `e` et la matrice est du type MP{Float64}.

On peut vouloir convertir les nombres Max-Plus en nombre de l'algèbre Min-Plus (où les signes des $-\infty$ sont inversés) bien que le type soit toujours du type Max-Plus :

In [15]:
p = minplus(MP(-Inf))
p, typeof(p)

(Inf, MP{Float64})

## Les constantes Max-Plus pour Julia

Certaines constantes (éléments neutres et absorbants que l'on verra plus tard) Max-Plus sont prédéfinies :
- mp0 pour $-\infty$ (que l'on note en général $\varepsilon$ ou parfois $\mathbb{0}$),
- mp1 pour 0 (que l'on note en général $e$ ou parfois $\mathbb{1}$),
- mptop pour $+\infty$ (utilisé pour l'algèbre Min-Plus).

In [ ]:
mp0, ϵ, mp1, mptop, mpzero(), mpone(), mptop

Le template peuvent être ajouté :

In [ ]:
mpzero(Int64), mpone(Int64), mpzero(Float64), mpone(Float64)

Mais attention mp0, mp1, $\epsilon$, `e` et mptop sont prédéfinis avec le type MP{Float}.

## Affichage des nombres Max-Plus

Il y a 4 styles possibles d'affichage des nombres Max-Plus que l'on peut contrôler avec la fonction `mp_change_display`, le mode $1$ étant celui défini par défaut :
- Style 0: les nombres $-\infty$ et les $0$ sont affichés tel quel.
- Style 1 ou 2: les nombres $-\infty$ sont affiché sous la forme d'un point.
- Style 3 ou 4: les nombres $-\infty$ sont affichés sous la forme $\varepsilon$.
- Style 1 ou 3: les zéros sont affichés $0$.
- Style 2 ou 4: les zéros sont affichés $e$.

Notons que les nombres réels qui peuvent être écrits comme des entiers seront affichés comme des entiers et que le style par défaut est le $1$ car il correspond à l'affichage de ScicosLab car ils permettent d'afficher les matrices creuses de façon compacte. En effet, il est commun en Max-Plus de devoir manipuler et afficher de grosses matrices creuses.

In [ ]:
mp_change_display(0)
I = MP([-Inf 0; 0 -Inf])

In [ ]:
mp_change_display(2)
I

In [ ]:
mp_change_display(3)
I

In [ ]:
mp_change_display(4)
I

In [ ]:
# Le mode par défaut :
mp_change_display(1)
I

A partir d'une matrice Max-Plus, on peut générer le code $\LaTeX$ grâce à la fonction `LaTeX` ou via la fonction `show` avec l'argument `MIME"text/latex"`. La fonction `mp_change_display` modifie en conséquence le code LaTeX généré.

In [ ]:
mp_change_display(0)
LaTeX(stdout, I)

Une fois ce code $\LaTeX$ compilé, il affichera :

$$\left[
\begin{array}{*{20}c}
-\infty & 0 \\
0 & -\infty \\
\end{array}
\right]$$

Alors que :

In [ ]:
mp_change_display(2)
LaTeX(stdout, I)

Une fois ce code $\LaTeX$ compilé, il affichera :

$$\left[
\begin{array}{*{20}c}
\varepsilon & e \\
e & \varepsilon \\
\end{array}
\right]$$

## Opérateur Max-Plus $\oplus$

L'opérateur addition redéfini par l'opérateur max() de l'algèbre classique. Son symbole, pour le différencier de l'addition dans l'algèbre classique, est $\oplus$. Mais en Julia on gardera le symbole `+`. Cet opérateur est associatif, commutatif, a un élément neutre (noté $\varepsilon$) et est idempotent.

$$a \oplus b \triangleq \max(a,b)$$

In [ ]:
mp_change_display(1);
a = MP(1.0); b = MP(3.0); c = MP(5.0);
(a, b, c)

In [ ]:
a + b    # ≜ max(a, b) == max(1, 3) == 3

#### Commutativité de $\oplus$

$$a \oplus b = b \oplus a$$
$$\triangleq$$
$$\max(a,b) = \max(b,a)$$

In [ ]:
a + b == b + a

#### Associativité de $\oplus$

$$a \oplus b \oplus c = (a \oplus b) \oplus c = a \oplus (b \oplus c)$$

In [ ]:
a + b + c == (a + b) + c == a + (b + c)

In [ ]:
a + b + c # ≜ max(a, b, c) == max(1, 3, 5)

#### Elément neutre $\varepsilon$ pour $\oplus$

$$a \oplus \varepsilon = \varepsilon \oplus a = a$$
$$\triangleq$$
$$\max(a,-\infty) = \max(-\infty,a) = a$$

In [ ]:
a + mp0 == mp0 + a == a

In [ ]:
a + ϵ == ϵ + a == a

In [ ]:
(a, mp0, ϵ), (a + mp0, a + ϵ), (mp0 + a, ϵ + a)

Notons que 0 est neutre pour les nombres positifs :

In [ ]:
a + 0 == 0 + a == a

In [ ]:
a, 0, a + 0

#### Elément absorbant $\infty$ pour $\oplus$

$$a \oplus \infty = \infty \oplus a$$
$$\triangleq$$
$$\max(a,\infty) = \max(\infty,a) = \infty$$

In [ ]:
a + mptop == mptop + a == mptop

In [ ]:
a, mptop, a + mptop

#### $\oplus$ est idempotent

In [ ]:
a + a    # ≜ max(a, a) == max(1, 1) == 1

## Opérateur Max-Plus $\otimes$

L'opérateur multiplication est redéfini par l'opérateur addition qui est associatif, commutatif, a l'élément neutre $e$, l'élément absorbant $\varepsilon$ et est distributif sur $\oplus$.

In [ ]:
a * b    # ≜ a + b == 1 + 3 == 4

#### Commutativité de $\otimes$

$$a \otimes b = b \otimes a$$
$$\triangleq$$
$$a + b = b + a$$

In [ ]:
a * b == b * a

#### Associativité de $\otimes$

$$a \otimes b \otimes c = (a \otimes b) \otimes c = a \otimes (b \otimes c)$$

In [ ]:
a * b * c == (a * b) * c == a * (b * c)

In [ ]:
a * b * c

#### Elément neutre $e$ pour $\otimes$

$$a \otimes e = e \otimes a = a$$
$$\triangleq$$
$$a + 0 = 0 + a = a$$

In [ ]:
a * mp1 == mp1 * a == a

In [ ]:
a * e == e * a == a

#### Elément absorbant $\varepsilon$ pour $\otimes$

$$a \otimes \varepsilon = \varepsilon \otimes a = \varepsilon$$
$$\triangleq$$
$$a -\infty = -\infty + a = -\infty$$

In [ ]:
a * mp0 == mp0 * a == mp0

In [ ]:
a * ϵ == ϵ * a == ϵ

Par convention:

$$\infty \otimes \varepsilon = \varepsilon \otimes \infty = \varepsilon$$

In [ ]:
# mptop * mp0 # FIXME shall return mp0

**FIXME** Help wanted

#### $\otimes$ n'est pas idempotent

In [ ]:
a * a    # ≜ a + a == 1 + 1 == 2

---
### Distributivité de $\otimes$ sur $\oplus$

$$a \otimes (b \oplus c) = (a \otimes b) \oplus (a \otimes c)$$
$$(b \oplus c) \otimes a = (b \otimes a) \oplus (c \otimes a)$$

$$a \oplus b \otimes c = a + \max(a, b)$$
$$a \otimes c \oplus b \otimes c = \max(a+c,b+c)$$

In [ ]:
(a + b) * c == (a * c) + (b * c)     # => max(a, b) + c == max(a + c, b + c) 

In [ ]:
(a * c) + (b * c)

## TODO

In [27]:
Q = MP([1.0 2; 3 4])

2×2 Matrix{MP{Float64}}:
  1.0   2.0
  3.0   4.0


In [28]:
Q .+ 2.0

2×2 Matrix{MP{Float64}}:
  2.0   2.0
  3.0   4.0


In [26]:
Q .* 2.0

2×2 Matrix{MP{Float64}}:
  3.0   4.0
  5.0   6.0


## Produit matriciel

Les matrices peuvent être de type Max-Plus. Le produit matriciel correspond au produit matriciel avec les opérateurs $+$ et $\times$ surchargés.

$$A=\begin{bmatrix}
4 & 3 \\
7 & -\infty
\end{bmatrix}\;,$$

$$A \otimes A = \begin{bmatrix}
4 \otimes 4 \oplus 3 \otimes7 & 4 \otimes 3 \oplus 3 \otimes -\infty \\
7 \otimes 4 \oplus -\infty \otimes 7 & 7 \otimes 3 \oplus -\infty \otimes -\infty
\end{bmatrix}\; = \begin{bmatrix}
10 & 7 \\
11 & 10
\end{bmatrix}\; = A^2.$$

In [21]:
A = MP([4 3; 7 -Inf])
A * A

2×2 Matrix{MP{Float64}}:
  10.0    7.0
  11.0   10.0


In [22]:
A * A == A^2

true

Fonctionne égallement sur les matrices creuses :

In [23]:
A * sparse(A) == sparse(A) * A == sparse(A) * sparse(A)

LoadError: UndefVarError: sparse not defined

En algèbre Max-Plus l'opérateur puissance se comporte comme une multiplication dans l'algèbre classique :

In [ ]:
MP(2.0)^5   # ==> 2 * 5

In [ ]:
MP(2.0)^0   # ==> 2 * 0

S'applique également aux matrices :

In [ ]:
A^5

In [ ]:
A^0

## Quelques matrice utiles

### Matrice dense d'identité

Taille 2 $\times$ 2 :

$$\left[
\begin{array}{*{20}c}
e & \varepsilon \\
\varepsilon & e \\
\end{array}
\right]$$

In [ ]:
I = mpeye(2,2) # Equivalent à : mpeye(Float64, 2,2)

In [ ]:
I = mpeye(3) # Equivalent à : mpeye(3,3)

Taille 3 $\times$ 2 :

In [ ]:
I = mpeye(3,2)

### Matrices denses remplies uniquement de $e$ :

$$\left[
\begin{array}{*{20}c}
e & e \\
e & e \\
\end{array}
\right]$$

In [ ]:
O = mpones(2,2) # Equivalent à : mpones(Float64, 2,2)

In [ ]:
O = mpones(2,3)

In [ ]:
O = mpones(2)

### Matrices creuses remplies de $\varepsilon$ :

$$\left[
\begin{array}{*{20}c}
\varepsilon & \varepsilon \\
\varepsilon & \varepsilon \\
\end{array}
\right]$$

In [ ]:
Z = mpzeros(2,2) # Equivalent à mpzeros(Float64, 2,2)

In [ ]:
Z = mpzeros(2,3)

In [ ]:
Z = mpzeros(2)

### Matrices denses remplies de $\varepsilon$ :

In [ ]:
Z = full(mpzeros(Float64, 2,2))

**Note :** On notera la présence de la fonction full() (ou son alias dense()) pour convertir une matrice creuse en matrice pleine (dense).

## Matrices creuses

Une matrice creuse est une matrice contenant beaucoup de zéros. En algèbre classique les zéros sont 0 (pour les entiers) ou 0.0 (réels) mais en Max-Plus les zéros valent $-\infty$. La structure interne des matrices creuses est conçu pour ne pas garder en mémoire ces zéros (sauf si demandé explicitement). Pour créer une matrice creuse en Julia il faut donner trois vecteurs : un vecteur pour garder les données non nulles et deux vecteurs pour mémoriser les index de ces données.

Beaucoup d'exemples dans la nature peuvent être représenté par des matrices creuses plutôt que par des matrices pleines comme par exemple le réseau routier est un graphe où les routes sont les arcs et les carrefours sont les noeuds. Les graphes sont représentés en mémoire soit sous forme de liste d'ajacence soit sous forme de matrice. Comme en général un carrefour permet de rejoindre entre 2 à 4 routes et que par conséquent jamais il n'existera une ville où tous les carrefours seront lés les uns aux autres il est donc préférable d'utiliser une matrice creuse. Mais à cause des leur index, les algorithmes sur les matrices seront plus pénalisés en temps d'exécution que pour les matrices pleines.

Pour la suite de ce document on aura besoin d'un paquet Julia de base supplémentaire :

In [ ]:
using SparseArrays

**Note: On préférera la version Julia > 1.3 qui corrige des bogues dans les matrices creuses. Les matrices creuses Max-Plus ne stockent pas les valeurs $-\infty$ mais dans les versions précédentes de Julia les valeurs zéros étaient écrites sous la forme littérale $0$ au lieu de la fonction zero() entraînant des cas d'erreurs. Le paquet MaxPlus en corrige certains mais il n'est pas garanti qu'il les corrige tous ou n'entraine pas des nouveaux !**

Création d'une matrice creuse de taille 2x2 Max-Plus vide :

In [ ]:
Z = mpzeros(2,2) # ou son équivalent mpzeros(Float64, 2,2)

Création d'une matrice creuse Max-Plus via des données non Max-Plus:

In [ ]:
MP(sparse([1, 2, 3], [1, 2, 3], [5, 2, 6]))

Création d'une matrice creuse Max-Plus avec des données Max-Plus :

In [ ]:
sparse([1, 2, 3], [1, 2, 3], MP([5, 2, 6]))

Julia autorise de créer des matrices stockant les éléments zéros qui ils sont explicitement donnés :

In [ ]:
A = MP(sparse([1, 2, 3], [1, 2, 3], [-Inf, 2, 0]))

Ici le -Inf n'est pas affiché mais il est bien stocké (3 stored entries).

In [ ]:
mp_change_display(0);
A.nzval      # nzval = Non-Zero values

Mais il n'est pas utilisé. Par exemple :

In [ ]:
B = MP(sparse([2, 3], [2, 3], [2.0, 0]))

In [ ]:
B.nzval

In [ ]:
A == B

Le $-\infty$ n'est pas testé. Si l'on ne désire pas stocker les $\epsilon$ on peut ajouter le paramètre `preserve=false` :

In [ ]:
C = MP(sparse([1, 2, 3], [1, 2, 3], [-Inf, 2, 0]), keepzeros=false)

In [ ]:
C.nzval

In [ ]:
A == B == C

### Conversion d'une matrice creuse en matrice pleine :

Les trois fonctions produisent le même résultat :

In [ ]:
full(Z), dense(Z), Array(Z)

In [ ]:
full(Z)

On remarquera que cette matrice contient que des $-\infty$. En effet, ils correspondent aux 0 éliminés des matrices creuses en algèbre classique. Une matrice creuse Max-Plus ne stocke pas les nombres Max-Plus $-\infty$ (**note:** enfin jusqu'à Julia > 1.3 car les versions précédentes avaient un bogue elles confondaient 0 et zero(T) avec T template de type MP). 

Conversion d'une matrice creuse Max-Plus à partir d'une matrice pleine en algèbre classique :

In [ ]:
A = [4.0 0; 7 5]

In [ ]:
B = mpsparse(A)

On remarquera que le $-\infty$ a disparu. Si on voulait le garder : **FIXME**

In [ ]:
mpsparse([4 0; 7 -Inf])

In [ ]:
C = sparse(MP(A))

In [ ]:
B.nzval, C.nzval

## Conversion de matrices Max-Plus

### Max-Plus vers Min-Plus

On peut vouloir convertir les valeurs pour l'algèbre Min-Plus (les signes des $-\infty$ sont inversés) pour les scalaires, matrices creuses et pleines :

In [ ]:
# Scalaires :
mp0, minplus(mp0)

In [ ]:
# Matrices creuses :
Z, minplus(Z)

In [ ]:
# Matrices pleines :
A = MP([4 0; 7 -Inf])
MP(A), minplus(MP(A))

### Max-Plus vers algèbre classique

On peut vouloir convertir une matrice Max-Plus en valeurs non Max-Plus :

In [ ]:
# Scalaires :
typeof(mp0), typeof(plustimes(mp0))

In [ ]:
plustimes(Z)

In [ ]:
plustimes(MP(A))

## Calcul sur les matrices
### Trace d'une matrice

La trace est la somme Max-Plus des éléments diagonaux.

In [ ]:
mptrace(A)

In [ ]:
mptrace(A) == A[1,1] + A[2,2]

### Norme d'une matrice

In [ ]:
mpnorm(mpeye(Float64, 2,2))

### Valeurs propres d'une matrice

In [ ]:
A = mpsparse([1.0 2; 3 4])
l,v = howard(A)

In [ ]:
A

In [ ]:
full(A)*v == l[1]*v

In [ ]:
A*v, full(A)*v # FIXME BUG JULIA

In [ ]:
full(A)*A

In [ ]:
A*full(A)

In [ ]:
A*A

---
### Algorithme $A^*$ (A star)